In [2]:
import numpy as np
import math

In [3]:
A = np.array([
    [12, -51, 4],
    [6, 167, -68],
    [-4, -24, -41]
])

In [4]:
ERROR = 0.000001

In [5]:
# Build the QR's Identity Matrix (Pij)
def buildPijMatrix(A, i, j):
    rows, cols = A.shape
    Pij = np.identity(rows)
    t_aux = np.arctan(A[i,j] / A[j,j])
    teta = (np.pi/4) if (np.abs(A[j,j]) < ERROR) else t_aux
    Pij[i,i] = np.cos(teta)
    Pij[j,j] = np.cos(teta)
    Pij[i,j] = - np.sin(teta)
    Pij[j,i] = np.sin(teta)
    
    return Pij

In [6]:
def QR_Decomposition(A):
    H = A.copy()
    rows, cols = H.shape
    Qt = np.identity(rows)
    for j in range(cols-1):
        for i in range(j+1, rows):
            Pt = buildPijMatrix(A, i, j)
            H = Pt@H
            Qt = Pt@Qt
    return Qt.T, H
Q, R = QR_Decomposition(A)
print(Q)
print(R)

[[ 0.84852814 -0.48290039  0.21634974]
 [ 0.42426407  0.86521406  0.26721644]
 [-0.31622777 -0.13495125  0.93903576]]
[[ 1.39928331e+01  3.51666309e+01 -1.24905057e+01]
 [-6.37153359e-02  1.72357498e+02 -5.52331566e+01]
 [ 4.43352546e-01  1.10544506e+01 -5.58057853e+01]]


## QR Decomposition

In [7]:
def QR_decomposition2(A):
    rows, cols = A.shape
    Q = np.empty((rows, rows))
    u = np.empty((rows, rows))
    
    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])
    
    for i in range(1, rows):
        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] #get each u vector
        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i])
    R = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(i, cols):
            R[i, j] = A[:, j] @ Q[:, i]
    return Q, R

Adjust the signs of the columns in Q and the rows in R

In [8]:
def diag_sign(A):
    return np.diag(np.sign(np.diag(A)))
def adjust_sign(Q, R):
    D = diag_sign(Q)
    Q[:, :] = Q @ D
    R[:, :] = D @ R
    return Q, R

In [9]:
Q, R = adjust_sign(*QR_decomposition2(A))

In [10]:
print(Q)
print(R)

[[ 0.85714286 -0.46732433  0.21659662]
 [ 0.42857143  0.88032235  0.20336934]
 [-0.28571429 -0.08148946  0.95484387]]
[[ 14.          34.71428571 -14.        ]
 [  0.         172.80312025 -58.39014935]
 [  0.           0.         -52.11132754]]


In [11]:
Q @ R

array([[ 12., -51.,   4.],
       [  6., 167., -68.],
       [ -4., -24., -41.]])

In [12]:
A

array([[ 12, -51,   4],
       [  6, 167, -68],
       [ -4, -24, -41]])

## QR method for eigenvalues and eigenvectors

In [14]:
def calc_error(A):
    soma = 0
    for i in range(1, A.shape[0]):
        for j in range(1, A.shape[1]):
            if(i!=j):
                soma += A[i,j]**2
    return math.sqrt(soma)

In [18]:
def QR(A, e, H):
    M = A.copy()
    rows, cols = M.shape
    Q = np.zeros((rows, 1))
    R = np.zeros((rows, 1))
    
    P = H
    MAX_INTERATION = 2000
    i = 0
    error = calc_error(M)
    while(error > e and i < MAX_INTERATION):
        Q, R = QR_Decomposition(M)
        M = R@Q
        P = P@Q
        error = calc_error(M)
        i += 1
    return M, P
eigen_values, eigen_vectors = QR(Q, ERROR, R)

In [19]:
print(eigen_values)

[[ 0.86461598 -0.29056123 -0.40989434]
 [ 0.20364059  0.94846502 -0.24278513]
 [ 0.45931439  0.12644478  0.87922807]]


In [20]:
print(eigen_vectors)

[[ -34.93407421   -5.26094751   18.68193039]
 [-157.15145303  -78.66748513   48.83825859]
 [ -38.72655663   21.77747971  -27.2320702 ]]
